In [4]:
import numpy as np
import ipywidgets as ipw
from IPython.display import Markdown, display, clear_output
from datetime import datetime
from utils import close_notebook, AppWidgets, read_json

In [ ]:
# Widgets
sample_preparation_dropdown = ipw.Dropdown(description='Sample Preparation Task', 
                                           disabled=False, 
                                           options = ["Sputtering", "Annealing", "Deposition", "Dosing"], 
                                           value = "Sputtering",
                                           layout = ipw.Layout(width = '400px'),
                                           style = {"description_width": '145px'})

In [5]:
def sample_preparation_widgets(change):
    
    clear_output()
    
    display(sample_preparation_dropdown)
    
    task = sample_preparation_dropdown.value
    
    if task != "Select a task...":
        app_widgets = AppWidgets(task, "config.json")

        # Populate dropdown lists
        app_widgets.load_dropdown_lists()

        display(Markdown(f"## {task}"))
        display(Markdown("### Select experiment, sample, and instrument"))
        
        display(app_widgets.select_experiment_output)
        
        with app_widgets.select_experiment_output:
            if task == "Deposition":
                display(app_widgets.experiments_dropdown_boxes, app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown_boxes, app_widgets.molecule_metadata_boxes)
            else:
                display(app_widgets.experiments_dropdown_boxes, app_widgets.sample_metadata_boxes, app_widgets.instruments_dropdown_boxes)
            
        app_widgets.create_button.on_click(app_widgets.create_process_action)
        app_widgets.quit_button.on_click(close_notebook)
        app_widgets.samples_dropdown.observe(app_widgets.load_sample_metadata, names = 'value')
        if task == "Deposition":
            app_widgets.molecules_dropdown.observe(app_widgets.load_molecule_metadata, names = 'value')

        display(Markdown("### Properties"))
        display(app_widgets.method_name_textbox, app_widgets.method_properties, app_widgets.comments_textbox)

        display(Markdown("### Support Files"))
        display(app_widgets.support_files_uploader)


        display(Markdown("### Save Sample"))
        app_widgets.method_name_textbox.observe(app_widgets.update_text, names = 'value')
        display(app_widgets.sample_out_name_textbox)
        display(app_widgets.bottom_buttons_hbox)
        display(app_widgets.increase_buttons_size)

# Sample Preparation

In [6]:
# Widgets
sample_preparation_dropdown.observe(sample_preparation_widgets, names='value')

# Trigger the callback manually
sample_preparation_widgets({'new': sample_preparation_dropdown.value})